In [ ]:
# import pandas and read in isobus log from a field
import pandas as pd

# specify url for combine ISOBUS file
url = "http://cloudradio39.ecn.purdue.edu/log/7130-f-14.log"

# we can give column definitions as we load in the file
_c = pd.read_csv(url, delimiter=' ', names=['ts', 'pgn', 'src', 'dest', 'pr', 'payload']);
_c.head()

In [ ]:
# we don't really need all the columns; to save time and space, we will only need the three columns
c = pd.read_csv(url, delimiter=' ', usecols=[0,1,5], names=['ts', 'pgn', 'payload']);
c.head()

In [ ]:
# we want to get only the engine load messsages
# so we will filter the `pgn` column by pgn number 61443
idx = c.index[c['pgn'] == 61443]
idx

In [ ]:
# now we will only take out the rows with indices we found
c_ = c.loc[idx]
c_.head()

In [ ]:
# remove extra index column
c_ = c_.reset_index().drop(columns=['index'])

In [ ]:
# now we want to create another column that contains only the 3rd byte which encodes the engine load value
c_['engine_load_hex']=c_.payload.str.slice(start=4, stop=6)
c_.head()

In [ ]:
# then we convert the hex value into decimal values and put them into a new column
c_['engine_load_percent'] = c_.engine_load_hex.apply(lambda x: int(x, 16))
c_.head()

In [ ]:
# the gps csv data only have gps timestamps in seconds
# we need the same thing here; that's why we round isobus timestamps to the nearest integers
import numpy as np

c_['ts_round'] = np.rint(c_.ts)
print(c_.ts_round.iat[0], c_.ts.iat[0])
c_.head()

In [ ]:
# since the rounded timestamps have duplicates, we need to downsample it
# the first step is to compute the engine load average for the same groups of timestamps
ts_groups = c_.groupby('ts_round')
engine_load_percent_mean = ts_groups.apply(lambda df: df['engine_load_percent'].mean())
engine_load_percent_mean

In [ ]:
# sanity check: do we have matching length arrays?
len(c_.ts_round.unique())

In [ ]:
# now we can create another data frame that contains the downsampled version of timestamp/engine load pair
d = {'gpsTimeSec': c_.ts_round.unique(), 'engine_load': engine_load_percent_mean.values}
e = pd.DataFrame(data=d)
e

In [ ]:
# now we can read in our gps csv
g = pd.read_csv('./data/gps/7130/7130-f-14.csv');
g.head()

In [ ]:
# note that the `gpsTime` has a weird notation, we need to preprocess that
g['gpsTimeSec'] = g.gpsTime / 1000
g.head()

In [ ]:
# merge two data frames by `gpsTimeSec`
eg = pd.merge(e, g, on='gpsTimeSec', how='inner')
eg.head()

In [ ]:
mapbox_token = open('_mapbox_token').readline().rstrip() # read in the mapbox token from file

import plotly.graph_objs as go
from plotly.offline import iplot


# define the color scale
scl = [0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],
[0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],
[0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]

data = []

data.append(
    go.Scattermapbox(                                                            
        lat=eg.lat,                                                              
        lon=eg.lon,                                                              
        mode='markers',                                                          
        marker=go.scattermapbox.Marker(                                          
            size=5,                                                              
            color=eg.engine_load,
            colorscale=scl,                                                      
            cmin=0,                                                              
            cmax=100,                                                              
            opacity=0.7,                                                         
            colorbar=dict(                                                       
                thickness=20,                                                    
                titleside="right",                                               
                outlinecolor="rgba(68, 68, 68, 0)",                              
                ticks="outside",                                                 
                ticklen=3,                                                       
                showticksuffix="last",                                           
                ticksuffix = " (%)",                                           
                dtick = 10
            )                                                                    
        )                                                                        
    )                                    
)

layout = go.Layout(                                                              
    title='Engine Load Map',                                     
    autosize=True,                                                               
    hovermode='closest',                                                         
    showlegend=False,                                                            
    mapbox=go.layout.Mapbox(                                                     
        accesstoken=mapbox_token,
        center=go.layout.mapbox.Center( # you can give a center coordinate of the figure
            lat=40.774773,                                                       
            lon=-102.284607,                                                     
        ), 
        bearing=0,                                                                                                                                 
        pitch=0,                                                                 
        zoom=9,                                                                 
        style='dark'                                                             
    )                                                                            
)           

fig = go.Figure(data=data, layout=layout)
iplot(fig)